(extendedkalmanfilter)=

## Extended Kalman Filter (EKF)
It is possible to do an exact parameter identification on perfect (simulated) data with no noise (see Section $\ref{\detokenize{06.40_results_inverse_dynamics::doc}}$). However, such data from physical experiments does not exist in reality. The measured data will always contain process noise and measurement noise. In order to mitigate this, the data is preprocessed using an Extended Kalman filter (EKF) and Rauch Tung Striebel (RTS) smoother which are both presented below.

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times"]})

from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]

In [ ]:
from wPCC_pipeline.pipelines.preprocess.nodes import filter, assemble_data, add_thrust
from wPCC_pipeline.pipelines.extended_kalman.nodes import create_extended_kalman
from wPCC_pipeline.pipelines.brix.nodes import initial_parameters
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter,extended_kalman_smoother
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import guess_covariance_matrixes, extended_kalman_filter, initial_state, extended_kalman_smoother
from wPCC_pipeline.pipelines.preprocess.nodes import load
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler

In [ ]:
#ship = "kvlcc2_hsva"
ship = "wpcc"
id = model_test_ids[ship][0]
#id = "22774"
raw_data = catalog.load(f"{ship}.{ id }.data")
ship_data = catalog.load(f"{ship}.ship_data")
vmm_name = "vmm_martins_simple"
vmm = catalog.load(vmm_name)
ek = catalog.load(f"{ ship }.{ vmm_name }.ek")
model = catalog.load(f"{ ship }.{ vmm_name }.ek")

data = load(raw_data, replace_velocities=True)

data_ek_smooth = catalog.load(f"{ ship }.updated.{ id }.data_ek_smooth")

### The EKF recursive algorithm
EKF is an extension of the Kalman Filter (KF) to work on nonlinear systems such as the manoeuvring models. The basic idea is that noise can be disregarded if it does not make sense from a physical point of view. If noisy measurement data were perfectly correct, this would mean that the ship has many vibrations that must have originated from tremendous forces, considering the large mass of the ship. The prior understanding of model tests suggests that these forces are not present during the test. Therefore, the noise should be considered as measurement noise and should be removed. Low-pass filtering is a common way to remove noise, where motions above some cut-off frequencies are regarded as unphysical measurement noise. The problem with low-pass filter is that it is hard to know what cut-off frequency to choose, either too low: removing part of the signal, or too high: keeping some unfiltered measurement noise in the data. The Kalman filter has a system model that continuously estimates the system's state that is run in parallel with the measurement data. The filter estimates the current state as a combination of the measurement data and the system model estimate based on belief in the data and the model. If the data has low noise, the estimate turns toward that data. Conversely, if the model gives very good predictions, then that estimate turns towards the model.

The system's inverse dynamics require the entire states, including positions, velocities, and accelerations, to be known. Only positions are known from the measurements, which means that velocities and accelerations are hidden states that the EKF should estimate.
The state transition $f(\mathbf{x},\mathbf{c})$ is taken from the manoeuvring model ([eq](eqf)) to use the manoeuvring model as the EKF predictor.
The state of the system is observed (measured) with a linear observation model ([eq](eqobserve)) where $\mathbf{y}$ is the measured data $\mathbf{H}$ is the observation matrix and $\eta$ is measurement noise.


```{math}
:label: eqobserve
\mathbf{y} = \mathbf{H} x + \eta
```

$$\mathbf{y} = \mathbf{H} \mathbf{x} + \eta $$

The used EKF recursive algorithm used is summarized in the pseudocode below $\cite{brown_introduction_1997}$.

```{prf:algorithm} Discrete-time extended Kalman filter
:label: ek-algorithm

**Inputs** Initial values: $x_0$, $P_0$, $C_d$, $R_d$, $Q_d$, $E_d$  

**Output** Estimated states: $\hat{x}$, estimated state covariances $\hat{P}$

1. Initial values:
    1. $\hat{x}[0] = x_0$ 
    2. $\hat{P}[0] = P_0$

2. For $k$ in $n$ measurements (time steps)

    1. KF gain
        1. $K[k]=\hat{P}[k] C_d^T \left(C_d \hat{P}[k] C_d^T + R_d\right)^{-1}$
        2. $I_{KC} = I_n - K[k] C_d$
        
    2. Update
        1. State corrector
            $\hat{x}[k] = \hat{x}[k] + K[k] (y - C_d \hat{x}[k]) $ 
        2. Covariance corrector
            $\hat{P}[k] = I_{KC} \cdot \hat{P}[k] I_{KC}^T + K[k] R_d K^T $
            
    3. Predict
        1. State predictor
            $\hat{x}[k+1] = \hat{x}[k] + h \cdot \hat{f}(\hat{x}[k], c[k])$
        2. Covariance predictor
            $\hat{P}[k+1] = A_d[k]  \hat{P}[k] A_d[k]^T + E_d Q_d E_d^T $
            

	
```

Where $n$ is number of states (6 in this case), $I_n$ is an $n \cdot n$ identity matrix.
The transition matrix is calculated for each iteration using a Jacobian of the transition model:
```{math}
:label: eqjacobi
A_d[k] = I + h \left. \frac{\partial f \left(x[k],c[k] \right)}{\partial x[k]} \right|_{x[k]=\hat{x}[k]}
```
This part and the fact that the nonlinear transition model is used directly as the predictor are the extension part of the EKF compared to the linear KF. Please note the linear approximation in [eq](eqjacobi) around the current state. This approximation can cause stability problems if the real system and the linearized system deviates too much, when large time steps are used on a very nonlinear system. The Unscented Kalman Filter, which was used in $\cite{revestido_herrero_two-step_2012}$, is an alternative that can be used in these situations. 


$$ A_d[k] = I + h \left. \frac{\partial f \left(x[k],c[k] \right)}{\partial x[k]} \right|_{x[k]=\hat{x}[k]} $$

The output from the filter contains the estimated states: $\hat{x}$ and estimated state covariance matrix $\hat{P}$. $\hat{x}$ represent the most likely estimates, but the estimates have uncertainty that is expressed in $\hat{P}$.
The state of the system is described by the ships position, heading, velocities and yaw velocity: 

```{math}
:label: eqstates
x = [x_0,y_0,\psi,u,v,r]^T
```
The initial state $x_0$ is taken as the mean value of the first five measurements, where the velocities are estimated with numeric differentiation. 

$$ x = [x_0,y_0,\Psi,u,v,r]^T $$

$C_d$ selects the measured states ($x_0$, $y_0$, $\psi$): 
```{glue:math} eqcd
:label: eqcd
```

In [ ]:
Cd = sp.symbols('C_d')
eq_Cd = sp.Eq(Cd, h*sp.UnevaluatedExpr(sp.Matrix(
        [
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
        ]
    )))

glue('eqcd',eq_Cd)

$E_d$ selects the hidden states ($u$, $v$, $r$): 

```{glue:math} eqed
:label: eqed
```

In [ ]:
Ed = sp.symbols('E_d')
eq_Ed = sp.Eq(Ed, h*sp.UnevaluatedExpr(sp.Matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1],
        ],
    )))

glue('eqed',eq_Ed)

Where $h$ is the discrete time step, $R_d$ describes the covariance matrix of the measurement, $Q_d$ is the covariance matrix of the process model, and $P_0$ is the initial state covariance.
Selecting good values for these three matrices is the most complicated part of getting the EKF to work well. The amount of expected measurement noise in the data should be inserted in to $R_d$, and the amount of error generated by the process model (manoeuvring model) needs to be estimated in $Q_d$. The choices for these matrices depend on the reliability of the present data and the present process model.

In [ ]:
ship='wpcc'
id = "22773"
vmm = "vmm_martins_simple"
data = catalog.load(f"{ship}.{id}.data")
ek = catalog.load(f"{ ship }.{ vmm }.ek")
ek_covariance_input = catalog.load(f"params:{ship}.updated.ek_covariance_input")
model = catalog.load(f"{ship}.updated.{vmm}.joined.model")

In [ ]:
ek_covariance_input

In [ ]:
x0 = initial_state(data=data)
covariance_matrixes = guess_covariance_matrixes(ek_covariance_input=ek_covariance_input, data=data)

In [ ]:
data2 = data.copy()
data2.iloc[0].update(x0)
result = model.simulate(data2)
df_sim = result.result

In [ ]:
data_noise = df_sim.copy()
data_noise.drop(columns=['u','v','r','u1d','v1d','r1d'], inplace=True)
np.random.seed(42)
data_noise['x0_epsilon']=np.random.normal(0, scale=20*ek_covariance_input['measurement_error_max']['positions']/3, size=len(data_noise))
data_noise['y0_epsilon']=np.random.normal(0, scale=20*ek_covariance_input['measurement_error_max']['positions']/3, size=len(data_noise))
data_noise['psi_epsilon']=np.random.normal(0, scale=20*np.deg2rad(ek_covariance_input['measurement_error_max']['psi']/3), size=len(data_noise))
data_noise['x0']+=data_noise['x0_epsilon']
data_noise['y0']+=data_noise['y0_epsilon']
data_noise['psi']+=data_noise['psi_epsilon']



In [ ]:

ek_filtered, data_ek_filter, time_steps = extended_kalman_filter(ek=ek, data=data_noise, covariance_matrixes=covariance_matrixes, x0=x0, hydrodynamic_derivatives=model.parameters, input_columns=['delta', 'thrust'])

(RTS)=
### The Rauch Tung Striebel (RTS) smoother
The EKF is recursive and can be run online, continuously making new estimates as new measurements arrive. The EKF uses passed measurements to estimate states in the near future. This prediction is helpful for online applications such as  autopilots or autonomous ships (USVs) (USVs). This restriction is  unnecessary for the parameter estimation on already existing data where a whole time series of existing measurements are available. The fact that both past and future data are known can be used to improve the filter. An EKF filter can include future time steps by adding a smoother after the filter. The parameter estimation uses a Rauch Tung Striebel (RTS) smoother $\cite{rauch_maximum_1965}$, which is an algorithm that runs the EKF backward to also account for future time steps.
The EKF and RTS have been run on simulated data with Gaussian noise added to see if the real states can be identified. Results from this can be seen in  {numref}`fig_ekf`. This figure shows that the RTS smoother is also needed to get an accurate estimate of the yaw acceleration.


```{glue:figure} fig_ekf
:figwidth: 1000px
:name: "fig_ekf"

EKF and RTS on simulated data (real) with Gaussian noise added (raw).
```

In [ ]:
ek_smooth, data_ek_smooth = extended_kalman_smoother(ek=ek_filtered, data=data_noise, time_steps=time_steps, covariance_matrixes=covariance_matrixes, hydrodynamic_derivatives=model.parameters)

In [ ]:
dataframes = {'raw':data_noise,
              'EKF':data_ek_filter,
              'RTS':data_ek_smooth,
              'real':df_sim, 
             }

styles = {
    'raw': {'style':'gray','lw':0.5},
    'real' : {'style':'b--', 'lw':1.5},
    'EKF' : {'style':'m-','alpha':1.0, 'lw':0.5},
    'RTS' : {'style':'g-','alpha':1.0, 'lw':1.5},

}

fig = plot(dataframes, keys=['psi','r','r1d'], styles=styles, ncols=1)
ax = fig.axes[0]
ax.set_xticks([])
ax.set_ylabel(r'$\psi$ [rad]')
ax = fig.axes[1]
ax.set_ylabel(r'$r$ [rad/s]')
ax.set_xticks([])
ax = fig.axes[2]
ax.set_xlabel('time [s]')
ax.set_ylabel(r'$\dot{r}$ $[rad/s^2]$')

plt.tight_layout()

glue('fig_ekf',fig, display=False)

In [ ]:
df_variance = pd.DataFrame(ek_filtered.variance.T, index= ek_filtered.time, columns=['x0','y0','psi','u','v','r'])

In [ ]:
fig,ax=plt.subplots()
df_variance.plot(ax=ax)
ax.set_ylim(0,0.3*df_variance.max().max());